In [1]:
%matplotlib inline

import requests
import pandas as pd 
import matplotlib.pyplot as plt
from taIndicators import basic, momentum

from pandas.plotting import scatter_matrix

from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import r2_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import train_test_split, time_series_split

import time

START_DATE = '2011-01-03'
END_DATE = '2019-04-03'
YEARLY_TRADING_DAYS = 252
MONTHLY_TRADING_DAYS = 21

ModuleNotFoundError: No module named 'taIndicators'

In [ ]:
# import files and convert to dataframe
file_name = "data/spy-momentum.csv"
df = pd.read_csv(file_name)

In [ ]:
df.set_index(['Symbol', 'Date'], inplace=True)

In [ ]:
df.drop(columns=['Pct_Change_Class','Rolling_Yearly_Mean_Price', 'Rolling_Monthly_Mean_Price', 'Rolling_Monthly_Mean_Positive_Days', 'Rolling_Yearly_Mean_Positive_Days', 'High', 'Low', 'Open', 'Close'], inplace=True)

In [ ]:
target_AdjClose = df.groupby(level=0)['AdjClose'].shift(-1 * MONTHLY_TRADING_DAYS)
target_return_rank_monthly = df.groupby(level=0)['Monthly_Return_Rank'].shift(-1 * MONTHLY_TRADING_DAYS)

In [2]:
df['target_AdjClose'] = target_AdjClose.values
df['target_return_rank'] = target_return_rank_monthly

NameError: name 'target_AdjClose' is not defined

In [3]:
df['target_isUp'] = df['target_AdjClose']
df['target_isTop100'] = df['target_return_rank']

NameError: name 'df' is not defined

In [524]:
topTile = 100
df['target_isUp'].where(df['target_AdjClose'] > df['AdjClose'], other=0, inplace=True)
df['target_isUp'].where(df['target_AdjClose'] < df['AdjClose'], other=1, inplace=True)
df['target_isTop100'].where(df['target_return_rank'] > topTile, other=1, inplace=True)
df['target_isTop100'].where(df['target_return_rank'] < topTile, other=0, inplace=True)

# # Run Simple Random Forest Classifier Model on the momentum dataset

In [525]:
ticker = 'AAPL' # choose ticker to run through the model

In [526]:
feature_list = ["AdjClose", "Pct_Change_Daily", "Pct_Change_Monthly", "Pct_Change_Yearly", "RSI", "Volatility",
               "Yearly_Return_Rank", "Monthly_Return_Rank", "Momentum_Quality_Monthly", "Momentum_Quality_Yearly",
               "SPY_Trailing_Month_Return"]
target_label_isTop100 = "target_isTop100"
target_label_isUp = "target_isUp"

In [527]:
df_stock = df.loc[ticker]
#df_stock.reset_index(inplace=True)
#df_stock.drop(columns=['Date'], inplace=True)
df_stock = df_stock.shift(-1 * MONTHLY_TRADING_DAYS)

In [528]:
df_stock.head()

,Volume,AdjClose,Pct_Change_Daily,Pct_Change_Monthly,Pct_Change_Yearly,RSI,Volatility,Yearly_Return_Rank,Monthly_Return_Rank,Momentum_Quality_Monthly,Momentum_Quality_Yearly,SPY_Trailing_Month_Return,target_AdjClose,target_return_rank,target_isUp,target_isTop100
Date,,,,,,,,,,,,,,,,
2011-01-03,64738800.0,32.799084,-0.002058,0.044755,NaN,59.505790,NaN,NaN,171.0,0.639358,NaN,0.027076,34.292721,154.0,1.0,0.0
2011-01-04,98449400.0,32.715263,-0.002556,0.036675,NaN,58.765410,NaN,NaN,219.0,0.174642,NaN,0.029926,33.850731,143.0,1.0,0.0
2011-01-05,80460100.0,33.006752,0.008910,0.037425,NaN,60.561643,NaN,NaN,214.0,0.178216,NaN,0.027499,33.888828,199.0,1.0,0.0
2011-01-06,121255400.0,33.519241,0.015527,0.054386,NaN,63.503939,NaN,NaN,188.0,0.776937,NaN,0.035953,33.575436,269.0,1.0,0.0
2011-01-07,95260200.0,33.835499,0.009435,0.056766,NaN,65.190855,NaN,NaN,204.0,0.810941,NaN,0.042709,33.022934,272.0,0.0,0.0


In [529]:
df_stock = pd.DataFrame(df_stock).fillna(0)

In [530]:
stock_features = df_stock[feature_list]
stock_labels_top100 = df_stock[target_label_isTop100]
stock_labels_isUp = df_stock[target_label_isUp]

In [531]:
def basic_random_forest_classifier(stock_features, stock_labels, test_size=.2):
    X_train, X_test, y_train, y_test = train_test_split(stock_features, stock_labels, test_size=test_size)
    model = RandomForestClassifier()
    model.fit(X_train, y_train)

    expected = y_test
    predicted = model.predict(X_test)

    print('Random Forest model')
    #print("Stock Percent Up Days: ", expected.mean(), "\n\n")
    print("Accuracy:",metrics.accuracy_score(expected, predicted))
    print(classification_report(expected, predicted))
    print(confusion_matrix(expected, predicted))

## Use Random Forest Classification to try to predict whether a stock's return will be ranked in the top 100 of the s&p 500

In [532]:
basic_random_forest_classifier(stock_features, stock_labels_top100)

Random Forest model
Accuracy: 0.8725961538461539
              precision    recall  f1-score   support

         0.0       0.88      0.96      0.92       304
         1.0       0.86      0.63      0.73       112

   micro avg       0.87      0.87      0.87       416
   macro avg       0.87      0.80      0.82       416
weighted avg       0.87      0.87      0.87       416

[[292  12]
 [ 41  71]]


/Users/georgekrug/anaconda3/envs/tenv/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


## Use Random Forest Classifcation to try to predict whether a stock will have a positive return in the next month

In [533]:
basic_random_forest_classifier(stock_features, stock_labels_isUp)

Random Forest model
Accuracy: 0.8485576923076923
              precision    recall  f1-score   support

         0.0       0.79      0.84      0.81       163
         1.0       0.89      0.85      0.87       253

   micro avg       0.85      0.85      0.85       416
   macro avg       0.84      0.85      0.84       416
weighted avg       0.85      0.85      0.85       416

[[137  26]
 [ 37 216]]


/Users/georgekrug/anaconda3/envs/tenv/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
